In [1]:
import pandas as pd
import os



In [2]:
# =======================
# 0. Definir ruta base del proyecto
# =======================
ruta_base = os.path.abspath(os.path.join(os.getcwd(), ".."))
ruta_silver = os.path.join(ruta_base, "SILVER")
ruta_gold = os.path.join(ruta_base, "GOLD")
os.makedirs(ruta_gold, exist_ok=True)


In [3]:
# =======================
# 1. Cargar archivos Parquet desde carpeta SILVER
# =======================
df_aceite = pd.read_parquet(os.path.join(ruta_silver, "aceite_condensado.parquet"))
df_gas = pd.read_parquet(os.path.join(ruta_silver, "gas_natural.parquet"))

In [15]:
# Definir el nombre de la columna de producción
columna_produccion = 'petroleo_mbd'

In [17]:
# Agrupar por 'campo', sumar la producción, ordenar y obtener el top 10
top_10_productores = df_aceite.groupby('campo')[columna_produccion].sum().sort_values(ascending=False).head(10)

In [19]:
print("\n--- Top 10 Campos por Producción Total de Petróleo ---")
print(top_10_productores)


--- Top 10 Campos por Producción Total de Petróleo ---
campo
MALOOB     37916.983854
ZAAP       27631.993049
XANAB       9817.470776
AYATSIL     7740.068869
KU          7419.903594
ONEL        5334.395431
XUX         4460.286633
AKAL        4336.547316
BALAM       4226.251500
YAXCHE      3296.676103
Name: petroleo_mbd, dtype: float64


In [23]:
# --- 3. Crear el nuevo DataFrame con la serie de tiempo del Top 10 ---

# Obtener la lista de nombres de los campos del top 10
lista_top_10_campos = top_10_productores.index.tolist()

In [27]:
# Filtrar el DataFrame original para mantener solo los campos del top 10
df_top_10 = df_aceite[df_aceite['campo'].isin(lista_top_10_campos)].copy()

In [29]:
 # --- 4. Preparar y guardar el DataFrame final ---
# Asegurarse de que la columna de fecha esté en formato datetime
df_top_10['fecha'] = pd.to_datetime(df_top_10['fecha'])

In [31]:
# Ordenar los datos para un análisis de series de tiempo correcto
df_top_10.sort_values(by=['campo', 'fecha'], inplace=True)

In [33]:
 # Guardar el resultado en un nuevo archivo CSV
ruta_archivo_salida = "top_10_campos_produccion_timeseries.csv"
df_top_10.to_csv(ruta_archivo_salida, index=False)
    

In [37]:
df_top_10.to_parquet(os.path.join(ruta_gold, "top_10_campos_timeseries.parquet"), index=False)

In [9]:
print("Vista previa aceite:")
display(df_aceite.head(2))


Vista previa aceite:


,fecha,cuenca,ubicacion,campo,operador,liquidos_mbd,petroleo_mbd,condensado_mbd
0,2016-01-01,CINTURON PLEGADO DE CHIAPAS,TERRESTRE,CERRO NANCHITAL,PEMEX EXPLORACIÓN Y PRODUCCIÓN,0.582716,0.582716,0.0
1,2016-01-01,CUENCAS DEL SURESTE,AGUAS SOMERAS,MAY,PEMEX EXPLORACIÓN Y PRODUCCIÓN,15.446212,15.446212,0.0


In [11]:
print("Vista previa gas:")
display(df_gas.head(2))


Vista previa gas:


,fecha,cuenca,ubicacion,campo,operador,gas_natural_sin_nitrogeno_mmpcd,nitrogeno_mmpcd
0,2016-01-01,BURGOS,TERRESTRE,VIERNES,PEMEX EXPLORACIÓN Y PRODUCCIÓN,0.199094,0.000192
1,2016-01-01,BURGOS,TERRESTRE,ALONDRA,PEMEX EXPLORACIÓN Y PRODUCCIÓN,3.698808,0.005928


In [13]:
df_aceite.describe()

,liquidos_mbd,petroleo_mbd,condensado_mbd
count,23126.000000,23126.000000,23126.000000
mean,8.227918,8.227663,0.000255
std,32.206100,32.206157,0.022025
min,0.000123,0.000123,0.000000
25%,0.242614,0.242426,0.000000
50%,1.161907,1.161539,0.000000
75%,4.863912,4.863912,0.000000
max,458.768492,458.768492,2.398355


In [15]:
# Ruta donde quieres guardar el archivo Excel
ruta_excel = os.path.join(ruta_gold, "aceite_limpio.xlsx")

# Exportar a Excel
df_aceite.to_excel(ruta_excel, index=False)

print(f"✅ Archivo exportado correctamente a: {ruta_excel}")

✅ Archivo exportado correctamente a: C:\Users\elezr\Datalquemy\Proyectos\CNH\GOLD\aceite_limpio.xlsx


In [16]:
# ===============================
#  Operadores únicos - Aceite
# ===============================
operadores_aceite = sorted(df_aceite["operador"].unique())
df_operadores_aceite = pd.DataFrame(operadores_aceite, columns=["operador"])
print(f"Total de operadores únicos en aceite: {len(operadores_aceite)}")
display(df_operadores_aceite)


Total de operadores únicos en aceite: 24


,operador
0,BLOQUE VC 01
1,CALICANTO OIL & GAS
2,CANAMEX ENERGY HOLDINGS
3,DEUTSCHE ERDOEL MÉXICO
4,DIAVAZ OFFSHORE
5,DS SERVICIOS PETROLEROS
6,ENI MÉXICO
7,FIELDWOOD ENERGY E&P MÉXICO
8,HOKCHI ENERGY
9,JAGUAR EXPLORACIÓN Y PRODUCCIÓN 2.3


In [17]:
# =======================
# . Agregar columna de año
# =======================
df_aceite["anio"] = pd.to_datetime(df_aceite["fecha"]).dt.year
df_gas["anio"] = pd.to_datetime(df_gas["fecha"]).dt.year


In [18]:
# =======================
# 3. Producción total por año
# =======================
df_total_anual_petroleo = df_aceite.groupby("anio")["petroleo_mbd"].sum().round(2).reset_index()
display(df_total_anual_petroleo.head())
df_total_anual_petroleo.to_parquet(os.path.join(ruta_gold, "total_anual_petroleo.parquet"), index=False)

df_total_anual_gas = df_gas.groupby("anio")["gas_natural_sin_nitrogeno_mmpcd"].sum().round(2).reset_index()
display(df_total_anual_gas.head())
df_total_anual_gas.to_parquet(os.path.join(ruta_gold, "total_anual_gas.parquet"), index=False)


,anio,petroleo_mbd
0,2016,25768.88
1,2017,23324.93
2,2018,21721.75
3,2019,20148.93
4,2020,19959.17


,anio,gas_natural_sin_nitrogeno_mmpcd
0,2016,57855.62
1,2017,48682.27
2,2018,44464.02
3,2019,45666.65
4,2020,46115.27


In [19]:
# =======================
# 4. Promedio por operador
# =======================
df_promedio_operador_petroleo = df_aceite.groupby("operador")["petroleo_mbd"].mean().round(2).reset_index()
display(df_promedio_operador_petroleo.sort_values(by="petroleo_mbd", ascending=False))
df_promedio_operador_petroleo.to_parquet(os.path.join(ruta_gold, "promedio_operador_petroleo.parquet"), index=False)

df_promedio_operador_gas = df_gas.groupby("operador")["gas_natural_sin_nitrogeno_mmpcd"].mean().round(2).reset_index()
display(df_promedio_operador_gas.sort_values(by="gas_natural_sin_nitrogeno_mmpcd", ascending=False))
df_promedio_operador_gas.to_parquet(os.path.join(ruta_gold, "promedio_operador_gas.parquet"), index=False)


,operador,petroleo_mbd
8,HOKCHI ENERGY,16.23
6,ENI MÉXICO,13.07
12,PEMEX EXPLORACIÓN Y PRODUCCIÓN,8.72
13,PERENCO MÉXICO,8.44
16,PETROFAC MÉXICO,7.04
7,FIELDWOOD ENERGY E&P MÉXICO,6.95
3,DEUTSCHE ERDOEL MÉXICO,5.28
23,WINTERSHALL DEA MÉXICO,4.12
17,PETROLERA CÁRDENAS MORA,2.64
19,SERVICIOS DE EXTRACCIÓN PETROLERA LIFTING DE M...,1.80


,operador,gas_natural_sin_nitrogeno_mmpcd
5,DEUTSCHE ERDOEL MÉXICO,14.59
9,ENI MÉXICO,12.92
20,PEMEX EXPLORACIÓN Y PRODUCCIÓN,12.01
10,FIELDWOOD ENERGY E&P MÉXICO,11.71
21,PERENCO MÉXICO,9.67
33,WINTERSHALL DEA MÉXICO,8.93
28,SERVICIOS MÚLTIPLES DE BURGOS,6.47
13,HOKCHI ENERGY,6.36
24,PETROFAC MÉXICO,5.98
25,PETROLERA CÁRDENAS MORA,5.89


In [25]:
df_promedio_operador_petroleo[df_promedio_operador_petroleo["operador"].str.contains("PEMEX", case=False, na=False)]


,operador,petroleo_mbd
12,PEMEX EXPLORACIÓN Y PRODUCCIÓN,8.72


In [27]:
df_promedio_operador_petroleo.sort_values(by="petroleo_mbd", ascending=False).head(10)


,operador,petroleo_mbd
8,HOKCHI ENERGY,16.23
6,ENI MÉXICO,13.07
12,PEMEX EXPLORACIÓN Y PRODUCCIÓN,8.72
13,PERENCO MÉXICO,8.44
16,PETROFAC MÉXICO,7.04
7,FIELDWOOD ENERGY E&P MÉXICO,6.95
3,DEUTSCHE ERDOEL MÉXICO,5.28
23,WINTERSHALL DEA MÉXICO,4.12
17,PETROLERA CÁRDENAS MORA,2.64
19,SERVICIOS DE EXTRACCIÓN PETROLERA LIFTING DE M...,1.80


In [29]:
# =======================
# 5. Top 10 campos productores
# =======================
df_top10_campos_petroleo = df_aceite.groupby(["campo", "cuenca", "operador"])["petroleo_mbd"].sum().round(2).reset_index().sort_values(by="petroleo_mbd", ascending=False).head(10)
display(df_top10_campos_petroleo)
df_top10_campos_petroleo.to_parquet(os.path.join(ruta_gold, "top10_campos_petroleo.parquet"), index=False)

df_top10_campos_gas = df_gas.groupby(["campo", "cuenca", "operador"])["gas_natural_sin_nitrogeno_mmpcd"].sum().round(2).reset_index().sort_values(by="gas_natural_sin_nitrogeno_mmpcd", ascending=False).head(10)
display(df_top10_campos_gas.head())
df_top10_campos_gas.to_parquet(os.path.join(ruta_gold, "top10_campos_gas.parquet"), index=False)

,campo,cuenca,operador,petroleo_mbd
164,MALOOB,CUENCAS DEL SURESTE,PEMEX EXPLORACIÓN Y PRODUCCIÓN,37916.98
329,ZAAP,CUENCAS DEL SURESTE,PEMEX EXPLORACIÓN Y PRODUCCIÓN,27631.99
317,XANAB,CUENCAS DEL SURESTE,PEMEX EXPLORACIÓN Y PRODUCCIÓN,9817.47
26,AYATSIL,CUENCAS DEL SURESTE,PEMEX EXPLORACIÓN Y PRODUCCIÓN,7740.07
149,KU,CUENCAS DEL SURESTE,PEMEX EXPLORACIÓN Y PRODUCCIÓN,7419.90
205,ONEL,CUENCAS DEL SURESTE,PEMEX EXPLORACIÓN Y PRODUCCIÓN,5334.40
324,XUX,CUENCAS DEL SURESTE,PEMEX EXPLORACIÓN Y PRODUCCIÓN,4460.29
9,AKAL,CUENCAS DEL SURESTE,PEMEX EXPLORACIÓN Y PRODUCCIÓN,4336.55
32,BALAM,CUENCAS DEL SURESTE,PEMEX EXPLORACIÓN Y PRODUCCIÓN,4226.25
327,YAXCHE,CUENCAS DEL SURESTE,PEMEX EXPLORACIÓN Y PRODUCCIÓN,3296.68


,campo,cuenca,operador,gas_natural_sin_nitrogeno_mmpcd
11,AKAL,CUENCAS DEL SURESTE,PEMEX EXPLORACIÓN Y PRODUCCIÓN,36323.62
556,XUX,CUENCAS DEL SURESTE,PEMEX EXPLORACIÓN Y PRODUCCIÓN,25303.19
419,QUESQUI,CUENCAS DEL SURESTE,PEMEX EXPLORACIÓN Y PRODUCCIÓN,23861.32
297,MALOOB,CUENCAS DEL SURESTE,PEMEX EXPLORACIÓN Y PRODUCCIÓN,22232.71
271,KU,CUENCAS DEL SURESTE,PEMEX EXPLORACIÓN Y PRODUCCIÓN,20198.75


In [31]:
# =======================
# 6. Operadores distintos
# =======================
df_operadores_distintos_petroleo = df_aceite[["operador"]].drop_duplicates()
display(df_operadores_distintos_petroleo)
df_operadores_distintos_petroleo.to_parquet(os.path.join(ruta_gold, "operadores_distintos_petroleo.parquet"), index=False)

df_operadores_distintos_gas = df_gas[["operador"]].drop_duplicates()
display(df_operadores_distintos_gas)
df_operadores_distintos_gas.to_parquet(os.path.join(ruta_gold, "operadores_distintos_gas.parquet"), index=False)


,operador
0,PEMEX EXPLORACIÓN Y PRODUCCIÓN
940,DIAVAZ OFFSHORE
950,SERVICIOS DE EXTRACCIÓN PETROLERA LIFTING DE M...
969,RENAISSANCE OIL CORP
970,CANAMEX ENERGY HOLDINGS
4547,PERSEUS TAJÓN
4903,VISTA ENERGY HOLDING II
4920,PETROFAC MÉXICO
5509,PETROLERA CÁRDENAS MORA
5539,DEUTSCHE ERDOEL MÉXICO


,operador
0,PEMEX EXPLORACIÓN Y PRODUCCIÓN
1579,CONSORCIO PETROLERO 5M DEL GOLFO
1587,CMM CALIBRADOR
1594,STRATA CPB
1611,DUNAS EXPLORACIÓN Y PRODUCCIÓN
1645,GRUPO MAREÓGRAFO
1764,DIAVAZ OFFSHORE
1776,SERVICIOS DE EXTRACCIÓN PETROLERA LIFTING DE M...
1797,RENAISSANCE OIL CORP
1798,CANAMEX ENERGY HOLDINGS


In [33]:
# =======================
# 7. Operador top 1 anual por volumen de gas
# =======================
df_operador_anual_gas = df_gas.groupby(["anio", "operador"])["gas_natural_sin_nitrogeno_mmpcd"].sum().reset_index()
df_top1_operador_anual_gas = df_operador_anual_gas.sort_values(["anio", "gas_natural_sin_nitrogeno_mmpcd"], ascending=[True, False]).groupby("anio").head(1)
display(df_top1_operador_anual_gas)
df_top1_operador_anual_gas.to_parquet(os.path.join(ruta_gold, "top1_operador_gas_anual.parquet"), index=False)



,anio,operador,gas_natural_sin_nitrogeno_mmpcd
7,2016,PEMEX EXPLORACIÓN Y PRODUCCIÓN,57598.868376
21,2017,PEMEX EXPLORACIÓN Y PRODUCCIÓN,48270.768210
42,2018,PEMEX EXPLORACIÓN Y PRODUCCIÓN,42944.266119
67,2019,PEMEX EXPLORACIÓN Y PRODUCCIÓN,43314.042605
94,2020,PEMEX EXPLORACIÓN Y PRODUCCIÓN,43413.500285
122,2021,PEMEX EXPLORACIÓN Y PRODUCCIÓN,43739.462145
150,2022,PEMEX EXPLORACIÓN Y PRODUCCIÓN,46210.967440
180,2023,PEMEX EXPLORACIÓN Y PRODUCCIÓN,48976.803210
211,2024,PEMEX EXPLORACIÓN Y PRODUCCIÓN,42840.098393
235,2025,PEMEX EXPLORACIÓN Y PRODUCCIÓN,3337.174176


In [35]:
# =======================
# 8. Operador top 1 anual por volumen de petróleo
# =======================
df_operador_anual_petroleo = df_aceite.groupby(["anio", "operador"])["petroleo_mbd"].sum().reset_index()
df_top1_operador_anual_petroleo = df_operador_anual_petroleo.sort_values(["anio", "petroleo_mbd"], ascending=[True, False]).groupby("anio").head(1)
display(df_top1_operador_anual_petroleo)
df_top1_operador_anual_petroleo.to_parquet(os.path.join(ruta_gold, "top1_operador_petroleo_anual.parquet"), index=False)


,anio,operador,petroleo_mbd
2,2016,PEMEX EXPLORACIÓN Y PRODUCCIÓN,25757.172059
6,2017,PEMEX EXPLORACIÓN Y PRODUCCIÓN,23300.652594
18,2018,PEMEX EXPLORACIÓN Y PRODUCCIÓN,21453.915934
34,2019,PEMEX EXPLORACIÓN Y PRODUCCIÓN,19691.158115
52,2020,PEMEX EXPLORACIÓN Y PRODUCCIÓN,19312.286542
70,2021,PEMEX EXPLORACIÓN Y PRODUCCIÓN,19208.248285
87,2022,PEMEX EXPLORACIÓN Y PRODUCCIÓN,18282.143399
106,2023,PEMEX EXPLORACIÓN Y PRODUCCIÓN,18617.247411
126,2024,PEMEX EXPLORACIÓN Y PRODUCCIÓN,17480.445798
140,2025,PEMEX EXPLORACIÓN Y PRODUCCIÓN,1334.404179


In [37]:
# Top 10 operadores por promedio de producción de petróleo
top10_operadores_aceite = (
    df_aceite.groupby("operador")["petroleo_mbd"]
    .mean()
    .round(2)
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
)

display(top10_operadores_aceite)



,operador,petroleo_mbd
0,HOKCHI ENERGY,16.23
1,ENI MÉXICO,13.07
2,PEMEX EXPLORACIÓN Y PRODUCCIÓN,8.72
3,PERENCO MÉXICO,8.44
4,PETROFAC MÉXICO,7.04
5,FIELDWOOD ENERGY E&P MÉXICO,6.95
6,DEUTSCHE ERDOEL MÉXICO,5.28
7,WINTERSHALL DEA MÉXICO,4.12
8,PETROLERA CÁRDENAS MORA,2.64
9,SERVICIOS DE EXTRACCIÓN PETROLERA LIFTING DE M...,1.80


In [39]:
# Guardar en Parquet
top10_operadores_aceite.to_parquet(
    os.path.join(ruta_gold, "top10_operadores_petroleo.parquet"), index=False
)

In [41]:
# Top 10 operadores por promedio de producción de gas
top10_operadores_gas = (
    df_gas.groupby("operador")["gas_natural_sin_nitrogeno_mmpcd"]
    .mean()
    .round(2)
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
)

display(top10_operadores_gas)

,operador,gas_natural_sin_nitrogeno_mmpcd
0,DEUTSCHE ERDOEL MÉXICO,14.59
1,ENI MÉXICO,12.92
2,PEMEX EXPLORACIÓN Y PRODUCCIÓN,12.01
3,FIELDWOOD ENERGY E&P MÉXICO,11.71
4,PERENCO MÉXICO,9.67
5,WINTERSHALL DEA MÉXICO,8.93
6,SERVICIOS MÚLTIPLES DE BURGOS,6.47
7,HOKCHI ENERGY,6.36
8,PETROFAC MÉXICO,5.98
9,PETROLERA CÁRDENAS MORA,5.89


In [43]:
# Guardar en Parquet
top10_operadores_gas.to_parquet(
    os.path.join(ruta_gold, "top10_operadores_gas.parquet"), index=False
)

In [45]:
# ================================
# Top operadores por producción total de ACEITE
# ================================
df_total_operador_aceite = (
    df_aceite.groupby("operador")["petroleo_mbd"]
    .sum()
    .round(2)
    .reset_index()
    .sort_values(by="petroleo_mbd", ascending=False)
)

print("🛢️ Top operadores por producción total de petróleo (Mbd):")
display(df_total_operador_aceite.head(10))

🛢️ Top operadores por producción total de petróleo (Mbd):


,operador,petroleo_mbd
12,PEMEX EXPLORACIÓN Y PRODUCCIÓN,184437.67
6,ENI MÉXICO,1438.13
16,PETROFAC MÉXICO,971.02
8,HOKCHI ENERGY,941.57
7,FIELDWOOD ENERGY E&P MÉXICO,534.80
17,PETROLERA CÁRDENAS MORA,438.01
5,DS SERVICIOS PETROLEROS,414.01
3,DEUTSCHE ERDOEL MÉXICO,364.49
13,PERENCO MÉXICO,286.95
19,SERVICIOS DE EXTRACCIÓN PETROLERA LIFTING DE M...,191.15


In [47]:
# Guardar en Parquet
df_total_operador_aceite.to_parquet(
    os.path.join(ruta_gold, "total_operador_aceite.parquet"), index=False
)

In [49]:
# ================================
# Top operadores por producción total de GAS
# ================================
df_total_operador_gas = (
    df_gas.groupby("operador")["gas_natural_sin_nitrogeno_mmpcd"]
    .sum()
    .round(2)
    .reset_index()
    .sort_values(by="gas_natural_sin_nitrogeno_mmpcd", ascending=False)
)

print("🔥 Top operadores por producción total de gas (MMpcd):")
display(df_total_operador_gas.head(10))

🔥 Top operadores por producción total de gas (MMpcd):


,operador,gas_natural_sin_nitrogeno_mmpcd
20,PEMEX EXPLORACIÓN Y PRODUCCIÓN,420645.95
28,SERVICIOS MÚLTIPLES DE BURGOS,6873.65
9,ENI MÉXICO,1421.40
4,CONSORCIO PETROLERO 5M DEL GOLFO,1116.19
5,DEUTSCHE ERDOEL MÉXICO,1006.48
25,PETROLERA CÁRDENAS MORA,978.32
10,FIELDWOOD ENERGY E&P MÉXICO,901.52
24,PETROFAC MÉXICO,819.59
19,PANTERA EXPLORACIÓN Y PRODUCCIÓN 2.2,648.30
26,RENAISSANCE OIL CORP,515.74


In [51]:
# Guardar en Parquet
df_total_operador_gas.to_parquet(
    os.path.join(ruta_gold, "total_operador_gas.parquet"), index=False
)

In [53]:
# ===============================
# 1. Operadores únicos - Aceite
# ===============================
operadores_aceite = sorted(df_aceite["operador"].unique())
df_operadores_aceite = pd.DataFrame(operadores_aceite, columns=["operador"])
print(f"Total de operadores únicos en aceite: {len(operadores_aceite)}")
display(df_operadores_aceite)


Total de operadores únicos en aceite: 24


,operador
0,BLOQUE VC 01
1,CALICANTO OIL & GAS
2,CANAMEX ENERGY HOLDINGS
3,DEUTSCHE ERDOEL MÉXICO
4,DIAVAZ OFFSHORE
5,DS SERVICIOS PETROLEROS
6,ENI MÉXICO
7,FIELDWOOD ENERGY E&P MÉXICO
8,HOKCHI ENERGY
9,JAGUAR EXPLORACIÓN Y PRODUCCIÓN 2.3


In [55]:
df_operadores_aceite.to_parquet(
    os.path.join(ruta_gold, "operadores_unicos_aceite.parquet"), index=False
)

In [57]:
# ===============================
# Campos únicos - Aceite
# ===============================
campos_aceite = sorted(df_aceite["campo"].unique())
df_campos_aceite = pd.DataFrame(campos_aceite, columns=["campo"])
print(f"Total de campos únicos en aceite: {len(campos_aceite)}")
display(df_campos_aceite)

Total de campos únicos en aceite: 304


,campo
0,ABKATUN
1,ACATIL
2,ACTUL
3,ACUATEMPA
4,AGAVE
...,...
299,YUM
300,ZAAP
301,ZACAMIXTLE
302,ZAMA


In [59]:
df_campos_aceite.to_parquet(
    os.path.join(ruta_gold, "campos_unicos_aceite.parquet"), index=False
)

In [61]:

# ===============================
# Operadores únicos - Gas
# ===============================
operadores_gas = sorted(df_gas["operador"].unique())
df_operadores_gas = pd.DataFrame(operadores_gas, columns=["operador"])
print(f"Total de operadores únicos en gas: {len(operadores_gas)}")
display(df_operadores_gas)

Total de operadores únicos en gas: 34


,operador
0,BLOQUE VC 01
1,CALICANTO OIL & GAS
2,CANAMEX ENERGY HOLDINGS
3,CMM CALIBRADOR
4,CONSORCIO PETROLERO 5M DEL GOLFO
5,DEUTSCHE ERDOEL MÉXICO
6,DIAVAZ OFFSHORE
7,DS SERVICIOS PETROLEROS
8,DUNAS EXPLORACIÓN Y PRODUCCIÓN
9,ENI MÉXICO


In [63]:
df_operadores_gas.to_parquet(
    os.path.join(ruta_gold, "operadores_unicos_gas.parquet"), index=False
)

In [65]:
# ===============================
# Campos únicos - Gas
# ===============================
campos_gas = sorted(df_gas["campo"].unique())
df_campos_gas = pd.DataFrame(campos_gas, columns=["campo"])
print(f"Total de campos únicos en gas: {len(campos_gas)}")
display(df_campos_gas)

Total de campos únicos en gas: 489


,campo
0,ABKATUN
1,ACAHUAL
2,ACATIL
3,ACTUL
4,ACUATEMPA
...,...
484,YUNQUE
485,ZAAP
486,ZACAMIXTLE
487,ZACATE


In [67]:
df_campos_gas.to_parquet(
    os.path.join(ruta_gold, "campos_unicos_gas.parquet"), index=False
)

In [69]:
# =======================
# 9. Guardado de datos limpios para ML
# =======================
display(df_aceite.head(2))
df_aceite.to_parquet(os.path.join(ruta_gold, "aceite_limpio.parquet"), index=False)
display(df_gas.head(2))
df_gas.to_parquet(os.path.join(ruta_gold, "gas_limpio.parquet"), index=False)


,fecha,cuenca,ubicacion,campo,operador,liquidos_mbd,petroleo_mbd,condensado_mbd,anio
0,2016-01-01,CINTURON PLEGADO DE CHIAPAS,TERRESTRE,CERRO NANCHITAL,PEMEX EXPLORACIÓN Y PRODUCCIÓN,0.582716,0.582716,0.0,2016
1,2016-01-01,CUENCAS DEL SURESTE,AGUAS SOMERAS,MAY,PEMEX EXPLORACIÓN Y PRODUCCIÓN,15.446212,15.446212,0.0,2016


,fecha,cuenca,ubicacion,campo,operador,gas_natural_sin_nitrogeno_mmpcd,nitrogeno_mmpcd,anio
0,2016-01-01,BURGOS,TERRESTRE,VIERNES,PEMEX EXPLORACIÓN Y PRODUCCIÓN,0.199094,0.000192,2016
1,2016-01-01,BURGOS,TERRESTRE,ALONDRA,PEMEX EXPLORACIÓN Y PRODUCCIÓN,3.698808,0.005928,2016


In [71]:
print("Años disponibles en df_aceite:", df_aceite["anio"].min(), "→", df_aceite["anio"].max())
print("Cantidad total de años distintos:", df_aceite["anio"].nunique())


Años disponibles en df_aceite: 2016 → 2025
Cantidad total de años distintos: 10


In [73]:
print("Lista de años únicos:", sorted(df_aceite["anio"].unique()))


Lista de años únicos: [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
